## Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- *оценка 2 и меньше* - низкий рейтинг
- *оценка 4.0 и меньше* - средний рейтинг
- *оценка 4.0 и 5* - высокий рейтинг

Результат классификации запишите в столбец *class*

In [1]:
import pandas as pd

data_ratings = pd.read_csv('ml-latest-small/ratings.csv')[['movieId', 'rating']]
data_ratings.head()

,movieId,rating
0,31,2.5
1,1029,3.0
2,1061,3.0
3,1129,2.0
4,1172,4.0


In [2]:
# Вариант 1
def classification(row):
    if row['rating'] <= 2:
        return 'низкий рейтинг'
    elif 2 < row['rating'] <= 4.0:
        return 'срений рейтинг'
    return 'высокий рейтинг'

# Вариант 2 (работает дольше)
# def classification(row):
#     if row['rating'] <= 2:
#         row['class'] = 'низкий рейтинг'
#     elif 2 < row['rating'] <= 4.0:
#         row['class'] = 'срений рейтинг'
#     else:
#         row['class'] = 'высокий рейтинг'
#     return row

In [3]:
# Вариант 1
data_ratings_classification = data_ratings.groupby('movieId').agg({'rating': 'mean'}).reset_index()
data_ratings_classification['class'] = data_ratings_classification.apply(classification,axis=1)
data_ratings_classification.head()
# Проверка
# len(data_ratings_classification)

# Вариант 2 (работает дольше)
# data_ratings.groupby('movieId').agg({'rating': 'mean'}).reset_index().apply(classification,axis=1).head()

,movieId,rating,class
0,1,3.872470,срений рейтинг
1,2,3.401869,срений рейтинг
2,3,3.161017,срений рейтинг
3,4,2.384615,срений рейтинг
4,5,3.267857,срений рейтинг


In [4]:
data_movies = pd.read_csv('ml-latest-small/movies.csv').sort_values(by='movieId')
data_movies.head()
# Проверка
# print(len(data_movies['movieId'].unique()))
# print(len(data_movies))


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
data_movies_extended = data_movies.set_index('movieId').join(data_ratings_classification.set_index('movieId')).reset_index()
data_movies_extended.head()

,movieId,title,genres,rating,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,срений рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,срений рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,срений рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,срений рейтинг
4,5,Father of the Bride Part II (1995),Comedy,3.267857,срений рейтинг


## Задание 2

Используем файл *keywords.csv*.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце *'region'* пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим *'undefined'*.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец *region*.

In [6]:
def geo_classificator(row):
    if ('москва' or 'тула' or'ярославль') in row['keyword'].lower():
        return 'Центр'
    elif ('петербург' or 'псков' or 'мурманск') in row['keyword'].lower():
        return 'Северо-Запад'
    elif ('владивосток' or 'сахалин' or 'хабаровск') in row['keyword'].lower():
        return 'Дальний Восток'
    else:
        return 'undefined'

In [7]:
data_requests = pd.read_csv('keywords.csv')
data_requests['region'] = data_requests.apply(geo_classificator,axis=1)
data_requests.head()
#Проверка
data_requests['region'].value_counts().head()

undefined         99462
Центр               284
Северо-Запад        211
Дальний Восток       43
Name: region, dtype: int64

## Задание 3

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файл *ratings.csv* c рейтингами фильмов, т.е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную *years* запишите список из всех годов с 1950 по 2010.
1. Напишите функцию *production_year*, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка *years*;
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска;
    - если ни один из номеров года списка *years* не встретился в названии фильма, то возвращается 1900 год.
1. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец *'year'*.
1. Посчитайте средний рейтинг всех фильмов для каждого значения столбца *'year'* и отсортируйте результат по убыванию рейтинга.

In [8]:
years = tuple(range(1950,2011))

In [9]:
def production_year(row):
    for year in years:
        if str(year) in row['title']:
            return year
    return 1900

In [10]:
data_movies_extended['year'] = data_movies_extended.apply(production_year,axis=1) #Взят готовый Framework из Задания 1
data_movies_extended.head(100)

,movieId,title,genres,rating,class,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,срений рейтинг,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,срений рейтинг,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,срений рейтинг,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,срений рейтинг,1995
4,5,Father of the Bride Part II (1995),Comedy,3.267857,срений рейтинг,1995
...,...,...,...,...,...,...
95,103,Unforgettable (1996),Mystery|Sci-Fi|Thriller,4.000000,срений рейтинг,1996
96,104,Happy Gilmore (1996),Comedy,3.292553,срений рейтинг,1996
97,105,"Bridges of Madison County, The (1995)",Drama|Romance,3.114583,срений рейтинг,1995
98,107,Muppet Treasure Island (1996),Adventure|Children|Comedy|Musical,3.462963,срений рейтинг,1996


In [31]:
result = data_movies_extended.groupby('year').agg({'rating': 'mean'}).loc[1:].sort_values(by='rating',ascending=False)
result.reset_index().head(20) #.loc[1:] - Исключение фильмов без даты (1900)

,year,rating
0,1951,3.906653
1,1952,3.802318
2,1962,3.772962
3,1953,3.738878
4,1957,3.693322
5,1973,3.688212
6,1971,3.643227
7,1969,3.604509
8,1972,3.585295
9,1958,3.571956
